In [13]:
## Create the clean files where 1 visit patients are removed

import pandas as pd

cvFolds = 10

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename = '/clinicalBERT/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extra = '_clean'
    extension = '_notext.csv'
    foldDf = pd.read_csv(filename+extension).drop(["Unnamed: 0"], axis=1)
    foldDfHadmCount = foldDf.groupby(['SUBJECT_ID'], as_index=False).agg(numAdmissions=('SUBJECT_ID', lambda x: len(x)))
    validSubjectIds = foldDfHadmCount[foldDfHadmCount.numAdmissions > 1].SUBJECT_ID
    cleanFoldDf = foldDf[foldDf.SUBJECT_ID.isin(validSubjectIds)].reset_index(drop=True)
    # print(cleanFoldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)) 
    # print(cleanFoldDf['Label'].value_counts())
    cleanFoldDf.to_csv(filename+extra+extension)
        

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [98]:
# READMISSION - Remove too early admissions in patients who have a lot of visits and keep only last 6

cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename1 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extension1 = '_clean_notext.csv'
    filename2 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_6visits_readmission_notext.csv'
   
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1)
    foldDf = foldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)
    lastVisits = foldDf.groupby(['SUBJECT_ID'], as_index=False).nth(-1)
    # Taking the last visit from the dataframe as we cannot use them for prediction here
    remainingVisitsDf = foldDf[~foldDf.HADM_ID.isin(lastVisits.HADM_ID)]
    last6Visits = remainingVisitsDf.groupby(['SUBJECT_ID']).tail(numberOfHistoryAdmissions)
    last6Visits.to_csv(filename2+extension2)
    
    # numAdmissions = remainingVisitsDf.groupby(['SUBJECT_ID'], as_index=False).agg(numAdmissions=('SUBJECT_ID', lambda x: len(x)))
    # print(max(numAdmissions.numAdmissions))
    # print(numAdmissions[numAdmissions.numAdmissions.isin(["8.0"])].SUBJECT_ID)
    # print(last6Visits['Label'].value_counts())


Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [115]:
# READMISSION - Structure data in a new matrix file where data is structured by patient and has the visit history organized as well as the label

cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    patientMatrix = []
    print("Fold Number {}:\n".format(i))
    filename1 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension1 = '_6visits_readmission_notext.csv'
    filename2 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_trajectory_readmission_notext.csv'
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1) 
    patientVisits = []
    prevSubject = ""
    prevLabel = ""
    for index in foldDf.index:
        subject_id = foldDf.loc[index, "SUBJECT_ID"]        
        if prevSubject != subject_id:
            while len(patientVisits)<7:
                patientVisits.append(0.0)
            if len(patientVisits) == 7:
                patientVisits.append(prevLabel)
            patientMatrix.append(patientVisits)
            patientVisits = []
            prevSubject = subject_id
            prevLabel = foldDf.loc[index, "Label"]
            patientVisits.append(subject_id)
            patientVisits.append(foldDf.loc[index, "HADM_ID"])
        elif prevSubject == subject_id:
            patientVisits.append(foldDf.loc[index, "HADM_ID"])
            prevLabel = foldDf.loc[index, "Label"]
    #Process and append last patient
    while len(patientVisits)<7:
        patientVisits.append(0.0)
    if len(patientVisits) == 7:
        patientVisits.append(prevLabel)
    patientMatrix.append(patientVisits)
    #Remove first dummy entry
    patientMatrix = patientMatrix[1:]
    # print(patientMatrix)
    patientMatrixDf = pd.DataFrame(patientMatrix, columns = ['SUBJECT_ID', 'VISIT_1', 'VISIT_2', 'VISIT_3', 'VISIT_4', 'VISIT_5', 'VISIT_6', 'Label'])
    patientMatrixDf.to_csv(filename2+extension2)
            
        
    
    

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [105]:
# DIAGNOSIS - Remove too early admissions in patients who have a lot of visits and keep only last 6
# The last visit was already removed priorly
cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    print("Fold Number {}:\n".format(i))
    filename1 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit/fold' + str(i)
    extension1 = '_codeprediction_notext.csv'
    filename2 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_6visits_diagnosis_notext.csv'
   
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1)  
    foldDf = foldDf.sort_values(['SUBJECT_ID','ADMITTIME'], ascending=True)
    last6Visits = foldDf.groupby(['SUBJECT_ID']).tail(numberOfHistoryAdmissions)
    # print(last6Visits[last6Visits.SUBJECT_ID.isin(["27172.0"])])
    last6Visits.to_csv(filename2+extension2)
    

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:



In [116]:
# DIAGNOSES - Structure data in a new matrix file where data is structured by patient and has the visit history organized as well as the label

cvFolds = 10
numberOfHistoryAdmissions = 6

for i in range(cvFolds):
    patientMatrix = []
    print("Fold Number {}:\n".format(i))
    filename1 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension1 = '_6visits_diagnosis_notext.csv'
    filename2 = '/clinicalBERT/data/extended_folds/discharge_subjectsplit_trajectory/fold' + str(i)
    extension2 = '_trajectory_diagnosis_notext.csv'
    foldDf = pd.read_csv(filename1+extension1).drop(["Unnamed: 0"], axis=1) 
    patientVisits = []
    prevSubject = ""
    prevLabelIcd = ""
    prevLabelCcs = ""
    for index in foldDf.index:
        subject_id = foldDf.loc[index, "SUBJECT_ID"]        
        if prevSubject != subject_id:
            while len(patientVisits)<7:
                patientVisits.append(0.0)
            if len(patientVisits) == 7:
                patientVisits.append(prevLabelIcd)
                patientVisits.append(prevLabelCcs)
            patientMatrix.append(patientVisits)
            patientVisits = []
            prevSubject = subject_id
            prevLabelIcd = foldDf.loc[index, "LABEL_NEXT_SMALL_DIAG_ICD9"]
            prevLabelCcs = foldDf.loc[index, "LABEL_NEXT_DIAG_CCS"]
            patientVisits.append(subject_id)
            patientVisits.append(foldDf.loc[index, "HADM_ID"])
        elif prevSubject == subject_id:
            patientVisits.append(foldDf.loc[index, "HADM_ID"])
            prevLabelIcd = foldDf.loc[index, "LABEL_NEXT_SMALL_DIAG_ICD9"]
            prevLabelCcs = foldDf.loc[index, "LABEL_NEXT_DIAG_CCS"]
    #Process and append last patient
    while len(patientVisits)<7:
        patientVisits.append(0.0)
    if len(patientVisits) == 7:
        patientVisits.append(prevLabelIcd)
        patientVisits.append(prevLabelCcs)
    patientMatrix.append(patientVisits)
    #Remove first dummy entry
    patientMatrix = patientMatrix[1:]
    # print(patientMatrix)
    patientMatrixDf = pd.DataFrame(patientMatrix, columns = ['SUBJECT_ID', 'VISIT_1', 'VISIT_2', 'VISIT_3', 'VISIT_4', 'VISIT_5', 'VISIT_6', 'LABEL_NEXT_SMALL_DIAG_ICD9', "LABEL_NEXT_DIAG_CCS"])
    patientMatrixDf.to_csv(filename2+extension2)


Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

Fold Number 5:

Fold Number 6:

Fold Number 7:

Fold Number 8:

Fold Number 9:

